# Real-Time Stream Viewer (HTTP)
the following function responds to HTTP requests with the list of last 10 processed twitter messages + sentiments in reverse order (newest on top), it reads records from the enriched stream, take the recent 10 messages, and reverse sort them. the function is using nuclio context to store the last results and stream pointers for max efficiency.<br> 

The code is automatically converted into a nuclio (serverless) function and and respond to HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations, and to deploy functions automatically onto a cluster.


## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that don't need to be copied to the function

In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [3]:
%%nuclio config 
# ask for a specific (fixed) API port
spec.triggers.web.kind = "http"
spec.triggers.web.attributes.port = 30100
# define the function base docker image
spec.build.baseImage = "python:3.6-jessie"

%nuclio: setting spec.triggers.web.kind to 'http'
%nuclio: setting spec.triggers.web.attributes.port to 30099
%nuclio: setting spec.build.baseImage to 'python:3.6-jessie'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [5]:
%nuclio cmd pip install v3io

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [3]:
import v3io.dataplane
import json
import os

def init_context(context):
    access_key = os.getenv('V3IO_ACCESS_KEY', None)
    v3io_client = v3io.dataplane.Client(endpoint='http://v3io-webapi:8081', access_key=access_key)
    resp = v3io_client.seek_shard(container='bigdata', path='stock_stream/0', seek_type='EARLIEST')
    context.next_location = resp.output.location
    context.data = []
    context.v3io_client = v3io_client

    
def handler(context, event):

    resp = context.v3io_client.get_records(container='bigdata', path='stock_stream/0', location=context.next_location, limit=10)
    context.next_location = resp.output.next_location
    context.logger.info('location: %s', context.next_location)

    for rec in resp.output.records:
        rec_data = rec.data.decode('utf-8')
        rec_json = json.loads(rec_data)
        context.data.append(rec_json)

    context.data = context.data[-10:]

    return json.dumps(context.data[::-1])
                            

In [ ]:
# nuclio: end-code

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [4]:
# nuclio: ignore
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
handler(context, event)

Python> 2020-08-16 07:47:57,629 [info] location: AQAAAAQAAAABAEBeGAAAAA==


'[{"content": "By Katie Paul\\nSAN FRANCISCO (Reuters) - Facebook  rolled out its own version of social media rival TikTok in the United States and more than 50 other countries on Wednesday, embedding a new short-form video service called Reels as a feature within its popular Instagram app.\\nThe product immediately got uptake with several celebrities, following a push by Facebook to attract creative talent before launch: actress Jessica Alba posted a video with her family promoting her Honest Company s masks, while comedian Mindy Kaling showed off an intentionally underwhelming quarantine \\"workout\\" routine.\\nReels  debut comes days after Microsoft  said it was in talks to acquire TikTok s U.S. operations from China s ByteDance. ByteDance has agreed to divest parts of TikTok, sources have said, under pressure from the White House, which has threatened to ban it and other Chinese-owned apps over data security concerns.\\nThe launch escalates a bruising fight between Facebook and Ti

## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [5]:
%nuclio deploy -p stocks -n stream-view

[nuclio] 2020-08-16 07:48:11,935 (info) Build complete
[nuclio] 2020-08-16 07:48:15,979 (info) Function deploy complete
[nuclio] 2020-08-16 07:48:15,986 done updating stream-view, function address: 192.168.224.185:30100
%nuclio: function deployed


In [1]:
# nuclio: ignore
# test the new API end point, take the address from the deploy log above
!curl 192.168.224.185:30100

[{"content": "By Yasin Ebrahim\nInvesting.com \u2013 Wall Street jumped on Wednesday as signs of progress on Capitol Hill over the next coronavirus fiscal stimulus package lifted investor sentiment.\nThe Dow Jones Industrial Average rose 1.24 %, or 331 points. The S&P 500 gained 0.66%, while the Nasdaq Composite jumped 0.52%.\nWith the clock ticking down on the congressional summer recess set for the end of this week, a sense of urgency to get the coronavirus relief package over the line has set in, with lawmakers on both sides said to have made modest concessions.\nSenate Majority Leader Mitch McConnell said on Tuesday that he was \"prepared to support\" federal unemployment benefits being brought back at that $600/week level \u2013 a key sticking point that has been holding up progress.\nValue stocks, which tend to perform well when the economy picks up, led the broader market higher, with materials, industrials, financials and energy up more than 1%.\nEnergy was rising on oil prices